In [27]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [28]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [29]:
spark

In [30]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
import fasttext
import re
from pyspark.sql.types import IntegerType,NumericType
import pandas as pd
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [31]:
emotes_baus =['SpongeWait', 'bausEZ', 'PETTHEBAUS', 'BRAMBLUS', 'bausPushups', 'KrugsUp', 'BausDonut', 'DaBaus', 'Babush', 'PPogo', 'FeelsHentai', 'SadgeJAM', 'Leanin', 'forsenCD', 'flashAbuser', 'Yoking', 'KKled', 'imposterSus', 'PepeLaffing', 'SirO', 'Svin', 'BABUS', 'PEPW', 'robLick', 'ezChamp', 'Diddles', 'BorpaFoundYou', 'Yoker', 'CLEAN', 'PepeUff', 'peepoChicken', 'vibePLS', 'Chatters', 'WEIRD', 'MODS', 'COCKA', 'DodgeAbusers', 'WeirdChamping', 'JoeBiden', 'TeaTime', 'Shilling', 'YEPCock', 'Weebs', 'IMustLean', 'ClubDance', 'BausBoomer', 'YEPJAM', 'PeterChamp', 'DONUT', 'batJAM', 'DESPAIR', 'lizardPls', 'bausL', 'YEBB', 'BOOBA', 'SadChamp', 'RIOT', 'IRELIAING', 'INSANECAT', 'Cummies', 'Duck', 'SCOOBA', 'Corpa', 'MUTE', 'Boomba', 'ABDULpls', 'MLADY', 'NODDERS', 'peepoHey', 'peepoTalk', 'FeelsWeakMan', 'TriBoom', 'PepegaDriving', 'gachiHYPER', 'Jammies', 'Nerdge', 'borpaSpin', 'KKool', 'peepoRasp', 'pepeMeltdown', 'Peeporiot', 'OOOO', 'PogO', 'PauseChamp', 'Tssk', 'furryRun', 'SION', 'gankAbuser', 'Gaspnt', 'modSheck', 'zhonyasAbuser', 'aniBlush', 'pepeP', 'NOOO', 'FLUSH', 'AYAYA', 'Haha', 'lickR', 'lickL', 'PagWoman', 'SionPassive', 'WeirdShamp', 'EatPoo', 'OMEGALOOOOL', 'Susge', 'Pepeg', 'poopoHappy']
emotes_adept = ['adeptTEA', 'mariRICH', 'peepoRun', 'gachiBASS', 'PepePls', 'Clap', 'pepeJAM', 'EZ', 'HYPERCLAP', 'PepegaPls', 'PepoDance', 'peepoPooPoo', 'pepeD', 'nymnCorn', 'pepeL', 'GuitarTime', 'gachiHYPER', 'ppOverheat', 'FeelsStrongJAM', 'sumSmash', 'COGGERS', 'forsenPls', 'ppHop', 'FeelsRainMan', 'xqcSlam', 'tenseSmash', 'CarlosPls', 'TranceGirls', 'FeelsLagMan', 'TriDance', 'PepegaAim', 'xqcOut', 'pepeMeltdown', 'TeaTime', 'peepoLeave', 'yikesJAM', 'peepoGiggles', 'peepoClap', 'WAYTOODANK', 'OuttaPocket', 'pepeCD', 'HACKERMANS', 'PepegaCredit', 'PepoG', 'PepeMods', 'DonoWall', 'PERIODT', 'BOOBA', 'Drake', 'modCheck', 'monkaSTEER', 'peepoShy', 'peepoFinger', 'PepeLa', 'Jammies', 'NOPERS', 'Chatting', 'Hmmm', 'Copium', 'Madge', 'Prayge', 'CLEAN', 'COWdance', 'DojaDance', 'Awkward', 'BLUBBERS', 'ppSmoke', 'peepoDJ', 'Susge', 'pugPls', 'peepoRiot', 'POLICE', 'lebronJAM', 'NODDERS', 'dojaJAM', 'MeganSus', 'catJAM', 'DinkDonk', 'GIGACHAD', 'HUHH', 'RIPBOZO', 'LOLOL', 'peepoBlonket', 'AlienPls', 'PepeFlushed', 'peepoTalk', 'peepoStir', 'Crying', 'oopTea', 'NOOO', 'pepeW', 'DIESOFCRINGE', 'Sussy', 'VeryPog', 'ppPoof', 'Bedge', 'ddHuh', 'Wokege', 'TomatoTime', 'catAwkward']


In [32]:
# df_result = globals()['my_model'].transform(df)
# df_result.show()

def identify_emotes_baus(text):
    return len([1 for word in emotes_baus if re.search(word, text)])
def identify_emotes_adept(text):
    return len([1 for word in emotes_adept if re.search(word, text)])

def preproc(msg):
    msg = msg.lower()
    msg = re.sub(r'[^\s\w]', '', msg)
    return msg
    
def Pred_probability(msg):
    msg = preproc(msg)
    model  = fasttext.load_model('ft.bin')
    pred = model.predict(msg.lower())[1]
    name = model.predict(msg.lower())[0][0]
    if name == '__label__thebausffs':
        pred = -pred
    else:
        pred = pred
    return float(pred)

def preproc(msg):
    msg = msg.lower()
    msg = re.sub(r'[^\s\w]', '', msg)
    return msg

def Predicted_Channel(x):
    if int(x) == 1:
        return 'adeptthebest'
    else:
        return 'thebausffs'
    return result

#predict_udf = udf(pred_chan,StringType())
preproc_udf = udf(preproc,StringType())
emotes_baus_udf =  udf(identify_emotes_baus, StringType())
emotes_adept_udf =  udf(identify_emotes_adept, StringType())
transform_probability_udf = udf(lambda z: Pred_probability(z)) 
Predicted_Channel_udf = udf(Predicted_Channel, StringType())
def process(time, rdd):
    if rdd.isEmpty():
        print('Empty')
        return
    
    print("========= %s =========" % str(time))
    # Convert to data frame
    df = spark.read.json(rdd)
    df = df.withColumn('msg', preproc_udf('message'))
    df = df.withColumn('Probability', transform_probability_udf('message').cast('float'))
    df = df.withColumn('AdeptEmote', emotes_adept_udf('message').cast('int'))
    df = df.withColumn('BausEmote', emotes_baus_udf('message').cast('int'))
    #df = df[['channel', 'msg','Probability','features','AdeptEmote','BausEmote']]
    assembler = VectorAssembler(inputCols=['BausEmote', 'AdeptEmote','Probability'],outputCol="features")
    df = assembler.transform(df)
    #df = df.withColumn('Features', spDF['features'])
    model = LogisticRegressionModel.load('notebooks\\RegModel')
    df = model.transform(df)
    df = df.withColumn('Predicted', Predicted_Channel_udf('prediction'))
    df = df[['channel','Predicted', 'message']]
    df.show()
    df.write.csv('Predictions', mode="append", header="true")

In [33]:
ssc = StreamingContext(sc, 5)

In [34]:
lines = ssc.socketTextStream("localhost", 8080)
lines.foreachRDD(process)

In [35]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

In [36]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
========= 2022-05-24 17:38:45 =========
+-----------+----------+--------------------+
|    channel| Predicted|             message|
+-----------+----------+--------------------+
|#thebausffs|thebausffs|@falendalle hate ...|
|#thebausffs|thebausffs|                Mots|
|#thebausffs|thebausffs|                  -2|
+-----------+----------+--------------------+

Empty
Empty
